## humidity (hus)

In [ ]:
# model #3 and #6 (r6i1p1, r5i1p1)
# model #3 and #6 (r2i1p1, r5i1p1)

In [ ]:
subset = intake.cat.nci['esgf'].cmip5.search(
                                            model_id = model, 
                                            experiment = experiment,
                                            time_frequency = 'day', 
                                            realm = 'atmos', 
                                            ensemble = ensemble, 
                                            variable= 'hus'
)
subset

In [ ]:
ds_dict = subset.to_dataset_dict()

In [ ]:
ds_hus = ds_dict[list(ds_dict.keys())[-1]].sel(time=period, lon=slice(0,360),lat=slice(-30,30))
ds_hus

In [ ]:
ds_hus.hus.encoding

In [ ]:
ds_hus = ds_hus.where(ds_hus.hus < 1e+20) 

In [ ]:
hus = regridder(ds_hus).hus*1000
hus.attrs['units']= 'g/kg'
hus

In [ ]:
# 100 g/kg are missing values (temp)

In [ ]:
hus_day=hus.isel(time=0).mean(dim='plev',keep_attrs=True)
lat = hus_day.lat
lon = hus_day.lon

In [ ]:
# plot snapshot
projection = ccrs.PlateCarree(central_longitude=180)

f, ax = plt.subplots(subplot_kw=dict(projection=projection), figsize=(15, 5))

hus_day.plot(transform=ccrs.PlateCarree(), cbar_kwargs={'orientation': 'horizontal','pad':0.125, 'aspect':50,'fraction':0.055},cmap='Greens')
ax.add_feature(cfeat.COASTLINE)
ax.set_extent([lon[0], lon[-1], lat[0], lat[-1]], crs=ccrs.PlateCarree())
ax.set_title('Snapshot, model:' + model + ' exp:' + experiment)
ax.set_xticks([-180, -90, 0, 90, 180])
ax.set_xticklabels([0, 90, 180, 270, 360])
ax.set_yticks([-20, 0, 20])
plt.tight_layout()

In [ ]:
hus_tMean= hus.mean(dim=('plev','time'), keep_attrs=True)

# areaweighting
# weights = np.cos(np.deg2rad(lat))
# weights.name = "weights"
# pr_tMean.weighted(weights).mean(dim=('lat','lon'))

In [ ]:
# plot timeMean
projection = ccrs.PlateCarree(central_longitude=180)

f, ax = plt.subplots(subplot_kw=dict(projection=projection), figsize=(15, 5))

hus_tMean.plot(transform=ccrs.PlateCarree(), cbar_kwargs={'orientation': 'horizontal','pad':0.125, 'aspect':50,'fraction':0.055}, cmap='Greens')
ax.add_feature(cfeat.COASTLINE)
ax.set_extent([lon[0], lon[-1], lat[0], lat[-1]], crs=ccrs.PlateCarree())
ax.set_title('tMean, model:' + model + ' exp:' + experiment)
ax.set_xticks([-180, -90, 0, 90, 180])
ax.set_xticklabels([0, 90, 180, 270, 360])
ax.set_yticks([-20, 0, 20])
plt.tight_layout()

In [ ]:
aWeights = np.cos(np.deg2rad(lat))
aWeights.name = "aWeights"

hus_sMean= hus.weighted(aWeights).mean(dim=('plev','lat','lon'), keep_attrs=True)

In [ ]:
# plot spatial mean
f, ax = plt.subplots(figsize=(15, 4))

hus_sMean.plot(ax=ax,label='spatial mean hus')


ax.set_title('sMean hus, model:' + model + ' exp:' + experiment)
#ax.set_xticks(rx1day_sMean.time.dt.year)

plt.tight_layout()
plt.legend()



## saving

In [ ]:
# humidity examples
save = False
if save:
    folder = '/g/data/k10/cb4968/cmip5/' + model
    os.makedirs(folder, exist_ok=True)
    
    fileName = model + '_hus_examples' + experiment + '.nc'
    path = folder + '/' + fileName
    if os.path.exists(path):
        os.remove(path)    

    xr.Dataset({'hus_day': hus_day, 'hus_tMean': hus_tMean}).to_netcdf(path)

In [ ]:
# humidity daily spatial mean
save = False
if save:
    folder = '/g/data/k10/cb4968/cmip5/' + model
    os.makedirs(folder, exist_ok=True)
    fileName = model + '_hus_daily' + experiment + '.nc'
    path = folder + '/' + fileName
    if os.path.exists(path):
        os.remove(path)    

    xr.Dataset({'hus_sMean': hus_sMean}).to_netcdf(path)